# Part 1. Sequence Tagging: NER

In [1]:
import pandas as pd
import os
import gensim.downloader
from gensim.models import Word2Vec
import tensorflow as tf
import numpy as np

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import InputLayer, TimeDistributed, SpatialDropout1D, Bidirectional
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout


from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from livelossplot import PlotLossesKeras


C:\Users\unkno\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\unkno\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


## 1.1 Word Embedding

In [2]:
w2v = gensim.downloader.load("word2vec-google-news-300")

### Qn 1.1

In [3]:
words = ["student", "Apple", "apple"]
print("-----------------------------------------------------------------------")
print("Word\t\tMost similar word\tCosine similarity")
print("-----------------------------------------------------------------------")
for word in words:
    most_similar = w2v.most_similar(positive=[word])
    print(f"{word}\t\t{most_similar[0][0]}  \t\t{most_similar[0][1]}")
print("-----------------------------------------------------------------------")

-----------------------------------------------------------------------
Word		Most similar word	Cosine similarity
-----------------------------------------------------------------------
student		students  		0.7294867038726807
Apple		Apple_AAPL  		0.7456986308097839
apple		apples  		0.720359742641449
-----------------------------------------------------------------------


## 1.2 Data

In [4]:
CoNLL2003_dir = '../Datasets/CoNLL2003_dataset'
train_dir = f'{CoNLL2003_dir}/eng.train'
dev_dir =  f'{CoNLL2003_dir}/eng.testa'
test_dir =  f'{CoNLL2003_dir}/eng.testb'

In [5]:
def import_content(path):
    try:
        with open(path, 'r') as file:
            content = file.readlines()
        file.close()
    except Exception as e:
        content = None
        print(e)
    
    return content

def print_items(item):
    for s in item: print(s)

In [6]:
train_content = import_content(train_dir)
dev_content = import_content(dev_dir)
test_content = import_content(test_dir)

### Split data by sentences

In [7]:
def split_sentences(content):
    split_data = [c.split(' ') for c in content] if content != None else []
    sentences = []
    sentence = []
    words = []

    for line in split_data:
        # if end of a sentence
        if line == ['\n']:
            sentences.append(sentence)
            sentence = []
        else:
            s_text  = line[0]
            s_tag = line[-1].replace('\n','')

            sentence.append([s_text, s_tag]) 
            words.append([s_text, s_tag])
    
    sentences.append(sentence) # last item in content not new line so must add previous sentence manually after loop           

    return sentences, words

In [8]:
def split_text_tag(sentences):
    text = []
    tag = []
    combined = []
    sentence_count = 1

    for s in sentences:
        for w in s:
            w_text  = w[0]
            w_tag = w[-1].replace('\n','')

            text.append(w_text)
            tag.append(w_tag)        
            combined.append({
                'sentence': sentence_count,
                'text' : w_text,
                'tag' : w_tag
            })   
        sentence_count+=1       
    return text, tag, combined

In [9]:
train_sentences, train_words = split_sentences(train_content)
dev_sentences, dev_words = split_sentences(dev_content)
test_sentences, test_words = split_sentences(test_content)

train_text, train_tag, train_combined = split_text_tag(train_sentences)
dev_text, dev_tag, dev_combined = split_text_tag(dev_sentences)
test_text, test_tag, test_combined = split_text_tag(test_sentences)

In [10]:
train_voc = np.unique(np.array(train_text))
dev_voc = np.unique(np.array(dev_text))


tag_set = np.unique(np.array(train_tag))

### Qn 1.2 (a)

#### Describe the size (number of sentences) of the training, development and test file for CoNLL2003.

In [11]:
print("Number of sentences (training):", len(train_sentences))
print("Number of sentences (dev):", len(dev_sentences))
print("Number of sentences (test):", len(test_sentences))

Number of sentences (training): 14987
Number of sentences (dev): 3466
Number of sentences (test): 3684


#### Specify the complete set of all possible word labels based on the tagging scheme (IO, BIO, etc.) you chose

In [12]:
print("Tag set (BIO):", tag_set)

Tag set (BIO): ['B-LOC' 'B-MISC' 'B-ORG' 'I-LOC' 'I-MISC' 'I-ORG' 'I-PER' 'O']


### Qn 1.2 (b)

#### Choose an example sentence from the training set of CoNLL2003 that has at least two named entities with more than one word.

In [13]:
def get_multiple_ne_sentence(sentences):
    for sentence in sentences:
        ne_count = 0
        for word_info in sentence:
            if "B-" in word_info[-1]:
                ne_count+=1
        if ne_count == 2:
            return sentence
    return None        

In [14]:
sentence = get_multiple_ne_sentence(train_sentences)
sentence

[['Swiss', 'I-MISC'],
 ['Grand', 'B-MISC'],
 ['Prix', 'I-MISC'],
 ['World', 'B-MISC'],
 ['Cup', 'I-MISC'],
 ['cycling', 'O'],
 ['race', 'O'],
 ['on', 'O'],
 ['Sunday', 'O'],
 [':', 'O']]

#### Explain how to form complete named entities from the label for each word, and list all the named entities in this sentence.

In [15]:
def get_named_entities(sentence):
    inside_tags = ['I-ORG', 'I-LOC', 'I-PER', 'I-MISC'] # Tags that require multiple words to form an entity
    begin_tags = ['B-LOC', 'B-ORG', 'B-MISC'] # Tags that are single word entities
    outside_tags = ['O']
    entities = [] # all entities gotten from search
    entity = [] # word group of current entity if any group tags encountered
    
    for c in sentence:
        if (c['tag'] in begin_tags or c['tag'] in outside_tags or c['tag'] == '\n') and len(entity) != 0:
            entities.append(' '.join(entity))
            entity = []
        if c['tag'] in begin_tags or c['tag'] in inside_tags: 
            entity.append(c['text'])

    return entities

In [16]:
_,_,sentence_text_tag = split_text_tag([sentence])
print("Complete named entities in the sentence:", get_named_entities(sentence_text_tag))

Complete named entities in the sentence: ['Swiss', 'Grand Prix', 'World Cup']


#### Tag-text dataset

In [17]:
train_df = pd.DataFrame(train_combined)
dev_df = pd.DataFrame(dev_combined)
test_df = pd.DataFrame(test_combined)

# path = '../Datasets/Processed/'
# file_name = 'CoNLL2003_processed'
# # Export DataFrame to a CSV file
# df.to_csv(f'{path}{file_name}.csv', index=False)

## 1.3 Model

### Create vocabulary index

In [18]:
#Load w2v models for train and dev

path = '../Pretrained_Models/'

train_w2v = Word2Vec.load('../Pretrained_Models/CONLL2003_pretrain.model')

train_pretrained_weights = train_w2v.wv.vectors
train_num_tokens, train_embedding_dim = train_pretrained_weights.shape

word2idx = train_w2v.wv.key_to_index
word2idx['<UNK>'] = word2idx[list(word2idx.keys())[-1]]+1
word2idx['<PAD>'] = word2idx[list(word2idx.keys())[-1]]+1
voc = train_voc
voc = np.append(voc,'<UNK>')
voc = np.append(voc,'<PAD>')

tag2idx = {k: v for v, k in enumerate(tag_set)}

### Gensim pre-trained word embeddings

In [19]:
w2v_word2idx = dict(w2v.key_to_index)
w2v_voc = w2v.index_to_key
w2v_word2idx['<UNK>'] = w2v_word2idx[list(w2v_word2idx.keys())[-1]]+1
w2v_word2idx['<PAD>'] = w2v_word2idx[list(w2v_word2idx.keys())[-1]]+1

### Create embedding matrix

In [20]:
embeddings_index = {}
for v in w2v_voc:
  embeddings_index[v] =  w2v[v] 


In [21]:
num_tokens = len(voc) + 2
embedding_dim = 50
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word2idx.items():    
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector[0:50]
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17724 words (5902 misses)


## LSTM

In [22]:
def get_x_sequence(sentences):
  sequence = []
  sent_seq = []
  for s in sentences:
    for w in s:
      if w[0] in word2idx.keys():
        sent_seq.append(word2idx[w[0]])
      else:
        sent_seq.append(word2idx['<UNK>'])
    sequence.append(sent_seq)
    sent_seq = []
  
  return sequence

In [41]:
x_train = get_x_sequence(train_sentences)
#x_train = pad_sequences(maxlen=embedding_dim, sequences=x_train, padding="post", value=len(word2idx)-1)
x_train = pad_sequences(maxlen=embedding_dim, sequences=x_train, padding="post")

y_train = [[tag2idx[w[1]] for w in s] for s in train_sentences]
y_train = pad_sequences(maxlen=embedding_dim, sequences=y_train, padding="post", value=tag2idx['O'])
#y_train = pad_sequences(maxlen=embedding_dim, sequences=y_train, padding="post")

x_dev = get_x_sequence(dev_sentences)
#x_dev = pad_sequences(maxlen=embedding_dim, sequences=x_dev, padding="post", value=len(word2idx)-1)
x_dev = pad_sequences(maxlen=embedding_dim, sequences=x_dev, padding="post")

y_dev = [[tag2idx[w[1]] for w in s] for s in dev_sentences]
y_dev = pad_sequences(maxlen=embedding_dim, sequences=y_dev, padding="post", value=tag2idx['O'])
#y_dev = pad_sequences(maxlen=embedding_dim, sequences=y_dev, padding="post")

x_test = get_x_sequence(test_sentences)
#x_dev = pad_sequences(maxlen=embedding_dim, sequences=x_dev, padding="post", value=len(word2idx)-1)
x_test = pad_sequences(maxlen=embedding_dim, sequences=x_test, padding="post")

y_test = [[tag2idx[w[1]] for w in s] for s in test_sentences]
y_test = pad_sequences(maxlen=embedding_dim, sequences=y_test, padding="post", value=tag2idx['O'])
#y_test = pad_sequences(maxlen=embedding_dim, sequences=y_test, padding="post")

y_train = to_categorical(y_train)
y_dev = to_categorical(y_dev)
y_test = to_categorical(y_test)

In [42]:
num_classes = len(tag_set)
sequence_length = 50
output_shape=(sequence_length,num_classes)

model = keras.Sequential()
model.add(InputLayer(embedding_dim))
#model.add(Embedding(input_dim=num_tokens, output_dim=embedding_dim,  embeddings_initializer=keras.initializers.Constant(embedding_matrix),
#    trainable=False,))
#model.add(SpatialDropout1D(0.01))
#model.add(Bidirectional(LSTM(units=embedding_dim, return_sequences=True, recurrent_dropout=0.1)))
model.add(Embedding(input_dim=num_tokens, output_dim=embedding_dim, input_length = sequence_length, trainable=False,))
model.add(LSTM(units=sequence_length))
model.add(Dense(units=256, activation= "relu", kernel_regularizer=regularizers.L1L2(l1=0.025, l2=0.025)))
model.add(Dropout(0.01))  
#model.add(Dense(units=256, activation= "sigmoid", kernel_regularizer=regularizers.L1L2(l1=0.0025, l2=0.0025)))
#model.add(Dropout(0.01))  
model.add(Dense(sequence_length * num_classes, activation='softmax'))
model.add(tf.keras.layers.Reshape(output_shape))

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
early_stopping = EarlyStopping(patience=10)
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 50)            1181400   
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense_4 (Dense)             (None, 256)               13056     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 400)               102800    
                                                                 
 reshape_2 (Reshape)         (None, 50, 8)             0         
                                                                 
Total params: 1317456 (5.03 MB)
Trainable params: 1360

In [43]:
num_epochs = 1000
batch_size = 1000

model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs, shuffle=True, validation_data=(x_dev, y_dev),
         callbacks = early_stopping, workers = 4)

Epoch 1/1000
15/15 [==============================] - 3s 115ms/step - loss: 24.2098 - accuracy: 0.7891 - val_loss: 21.2331 - val_accuracy: 0.9415
Epoch 2/1000
15/15 [==============================] - 1s 91ms/step - loss: 18.4657 - accuracy: 0.9485 - val_loss: 15.6712 - val_accuracy: 0.9510
Epoch 3/1000
15/15 [==============================] - 1s 91ms/step - loss: 13.7213 - accuracy: 0.9546 - val_loss: 11.6970 - val_accuracy: 0.9510
Epoch 4/1000
15/15 [==============================] - 1s 91ms/step - loss: 10.1175 - accuracy: 0.9546 - val_loss: 8.4604 - val_accuracy: 0.9510
Epoch 5/1000
15/15 [==============================] - 1s 93ms/step - loss: 7.1722 - accuracy: 0.9546 - val_loss: 5.8330 - val_accuracy: 0.9510
Epoch 6/1000
15/15 [==============================] - 1s 93ms/step - loss: 4.8189 - accuracy: 0.9546 - val_loss: 3.7822 - val_accuracy: 0.9510
Epoch 7/1000
15/15 [==============================] - 1s 95ms/step - loss: 3.0132 - accuracy: 0.9546 - val_loss: 2.2530 - val_accuracy

In [45]:
predicted_labels = model.predict(x_test)
print(predicted_labels)

116/116 [==============================] - 0s 3ms/step
[[[6.15498436e-07 5.82216671e-07 5.58322256e-07 ... 1.62139512e-03
   8.70085030e-04 5.98045159e-03]
  [3.87648561e-06 4.49454683e-06 5.40902988e-07 ... 7.84689386e-04
   1.06217689e-03 7.01039424e-03]
  [1.17958211e-06 6.08354355e-07 5.34358151e-07 ... 5.33043756e-04
   8.11943668e-04 7.43896794e-03]
  ...
  [1.69001646e-06 1.67680071e-06 1.64462256e-06 ... 1.25277775e-05
   1.78473674e-05 4.47417572e-02]
  [1.82357815e-06 1.56989813e-06 1.54706765e-06 ... 6.67803488e-06
   1.24740336e-05 4.55944128e-02]
  [2.34641379e-06 2.11114525e-06 2.30248975e-06 ... 2.12997679e-06
   1.74081561e-06 1.11727633e-01]]

 [[6.15498436e-07 5.82216671e-07 5.58322256e-07 ... 1.62139512e-03
   8.70085030e-04 5.98045159e-03]
  [3.87648561e-06 4.49454683e-06 5.40902988e-07 ... 7.84689386e-04
   1.06217689e-03 7.01039424e-03]
  [1.17958211e-06 6.08354355e-07 5.34358151e-07 ... 5.33043756e-04
   8.11943668e-04 7.43896794e-03]
  ...
  [1.69001646e-06 1.67

In [46]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

116/116 [==============================] - 0s 3ms/step - loss: 0.2315 - accuracy: 0.9565
Test Loss: 0.23151466250419617, Test Accuracy: 0.9565255045890808


In [148]:
x_train[0], len(x_train[0])

(array([  973, 12289,   237,   783,     5,  4433,   211,  6498,     0,
        23625, 23625, 23625, 23625, 23625, 23625, 23625, 23625, 23625,
        23625, 23625, 23625, 23625, 23625, 23625, 23625, 23625, 23625,
        23625, 23625, 23625, 23625, 23625, 23625, 23625, 23625, 23625,
        23625, 23625, 23625, 23625, 23625, 23625, 23625, 23625, 23625,
        23625, 23625, 23625, 23625, 23625]),
 50)

In [935]:
y_train[i], len(y_train[i])

(array([5, 7, 4, 7, 7, 7, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7]),
 50)

In [149]:
predicted_labels_sentence, len(predicted_labels_sentence)

(array([1.1694952e-01, 4.9378670e-08, 2.4964581e-01, 0.0000000e+00,
        0.0000000e+00, 4.4022022e-20, 7.8977473e-20, 1.3015374e-04,
        1.1669247e-37, 2.9517738e-17, 6.3284081e-01, 3.5020872e-15,
        4.2665351e-04, 9.5816960e-32, 1.2303811e-29, 4.0297813e-25,
        0.0000000e+00, 9.8721791e-38, 0.0000000e+00, 3.9840713e-36,
        6.9979742e-06, 1.3861541e-22, 3.6760278e-37, 1.6089272e-18,
        1.5299677e-28, 4.0904094e-28, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        4.3691262e-28, 7.8219448e-29, 3.4032821e-17, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00], dtype=float32),
 50)